In [1]:
import os
os.chdir("..")
from parc import model
from parc import IO
from parc import losses
from parc import graphs
import keras

#initialize input data characteristics
input_img_size = 485  #image size
case_numbers = 2     #number of cases
time_steps = 20       #number of time steps
del_t = 0.79*(10**-9) #time step size

#initialize adaptable model parameters
input_size = (None,None,2)
depth = 3
kernel_size = 5
numFeatureMaps = 128

#initialize loss parameters
weight_loss = [4,1,1] #initial timestep weights, middle weights, late weights

#Parse the raw data and return microstructure data and temperature/pressure outputs
data_in, total_output = IO.parse_data("data/raw",input_img_size,case_numbers,time_steps,del_t)

#Split data into training, validation, and testing sets
X_train, y_train, X_val, y_val, test_X, test_Y = IO.split_data(data_in,total_output,splits=[0.5, 0, 0.5])

#initilize model and model parameters
parc_model = model.parc(input_size,time_steps,depth,kernel_size,numFeatureMaps)
loss = ["mse", "mse"]
loss_weights = [1, 1]
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
metrics = ["mae"]
parc_model.compile(loss = loss,loss_weights = loss_weights,optimizer = optimizer,metrics = metrics)

########for 5 dimensional data format############
#x_tn_init = y_train[:,:,:,0,:1]
#x_val_init = y_val[:,:,:,0,:1]

#train the model using training set 
#history = parc_model.fit(
#    x=[X_train, x_tn_init],
#    y=[y_train[:, :, :, 1:, :1], y_train[:, :, :, 1:, 2:3]],
#    validation_data=([X_val, x_val_init], [y_val[:, :, :, :, :1], y_val[:, :, :, :, 2:3]]),
#    batch_size=1,
#    epochs=2,
#)
################################################

#####for 4 dimensional data format#######
#reshape to match old data format
y_train = IO.reshape_old(y_train)
y_val = IO.reshape_old(y_val)
test_Y = IO.reshape_old(test_Y)

x_tn_init = y_train[:,:,:,0:2]
x_val_init = y_val[:,:,:,0:2]
x_test_init = test_Y[:,:,:,0:2]

#train the model using training set, skip this part if already trained
#history = parc_model.fit(
#    x=[X_train, x_tn_init],
#    y=[y_train[:, :, :, 2:40], y_train[:, :, :, 40:]],
#    validation_data=([X_val, x_val_init], [y_val[:, :, :, 2:40], y_val[:, :, :, 40:]]),
#    batch_size=1,
#    epochs=2,
#)

#save trained model weights
parc_model.save_weights("PARC_trained_data.h5")

#load trained model weights
parc_model.load_weights("PARC_trained_data.h5")

#prediction using trained model parameters
pred = parc_model.predict([test_X,x_test_init])

#definition of temperature prediction and ground truth with rescaling
Temp_pred = pred[0][:,:,:,0::2]
Temp_pred = Temp_pred[:,:,:,:19]
Temp_pred = (Temp_pred+1.0)/2.0
Temp_gt = test_Y[:,:,:,0::2]
Temp_gt = Temp_gt[:,:,:,:19]
Temp_gt = (Temp_gt+1.0)/2.0

#definition of pressure prediction and ground truth with rescaling
Pres_pred = pred[0][:,:,:,1::2]
Pres_pred = Pres_pred[:,:,:,:19]
Pres_pred = (Pres_pred+1.0)/2.0
Pres_gt = test_Y[:,:,:,1::2]
Pres_gt = Pres_gt[:,:,:,:19]
Pres_gt = (Pres_gt+1.0)/2.0

#calculate losses
pred_cases = pred[0].shape[0]
rmse = losses.rmse(Temp_gt,Temp_pred,pred_cases,time_steps)
r2 = losses.r2(Temp_gt,Temp_pred,pred_cases,time_steps)
loss_cv = losses.step_weighted_loss(Temp_gt,Temp_pred, weight_loss)
step_loss = losses.step_weighted_physical_loss(Temp_gt,Temp_pred,loss_cv)
state_loss = losses.state_weighted_loss(Temp_gt,Temp_pred)

#reshape to match new data format
test_Y = IO.reshape_new(test_Y)
pred[0] = IO.reshape_new(pred[0],2)
pred[0][:,:,:,0,0] = test_Y[:,:,:,0,0]
pred[0][:,:,:,0,1] = test_Y[:,:,:,0,1]

#create plots for data
graphs.plot_rmse(rmse)
graphs.plot_r2(r2)

max and min of channel 0 are:  4000.0 300.0
max and min of channel 1 are:  33671716000.0 -1553485500.0
max and min of channel 2 are:  4683544303797.468 -4242651063291.1387
max and min of channel 3 are:  4.262242531645571e+19 -3.252435227848101e+19
Finished Processing Data
shape of microstructure data is:  (2, 240, 240, 2)
shape of output data is:  (2, 240, 240, 20, 4)
2
1
1
2
(1, 240, 240, 2)
(1, 240, 240, 20, 4)
(0, 240, 240, 2)
(0, 240, 240, 20, 4)
(1, 240, 240, 2)
(1, 240, 240, 20, 4)
Starting shape of data:  (1, 240, 240, 20, 4)
Reformatted data shape:  (1, 240, 240, 78)
Starting shape of data:  (0, 240, 240, 20, 4)
Reformatted data shape:  (0, 240, 240, 78)
Starting shape of data:  (1, 240, 240, 20, 4)
Reformatted data shape:  (1, 240, 240, 78)
1/1 [==============================] - 41s 41s/step
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Austin Leonard\Documents\Anaconda\envs\parc\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Austin Leonard\AppData\Local\Temp\ipykernel_24200\2399046848.py", line 101, in <cell line: 101>
    step_loss = losses.step_weighted_physical_loss(Temp_gt,Temp_pred,loss_cv)
  File "C:\Users\Austin Leonard\parc\parc\losses.py", line 141, in step_weighted_physical_loss
  File "C:\Users\Austin Leonard\Documents\Anaconda\envs\parc\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\Austin Leonard\Documents\Anaconda\envs\parc\lib\site-packages\tensorflow\python\framework\ops.py", line 7209, in raise_from_not_ok_status
    raise core._status_to_exception(e) from None  # pylint: disable=protected-access
tensorflow.python.framework.errors_impl.InvalidA